# Guide
실행 불필요  
전처리 완료된 데이터 링크 아래 참고

---

# Data
[[Google Drive] /Data/Final/Step1.zip](https://drive.google.com/file/d/1W6cyqB21ewVnygA3lyKvNsQI0IxAEYFb/view?usp=sharing)

## Dataset Source
- [[Kaggle] Fruit and Vegetable Disease (Healthy vs Rotten)](https://www.kaggle.com/datasets/muhammad0subhan/fruit-and-vegetable-disease-healthy-vs-rotten)  
apple 데이터만 사용  
- [[Roboflow] apples Computer Vision Project](https://universe.roboflow.com/ds-lxa2d/apples-daz2v)  
프로젝트에 사용된 데이터 사용

## Preprocessing
- Kaggle  
기존 데이터셋에는 이미 증강 이미지 존재(일부 클래스에만) → 다른 Step의 모델에도 사용하기 위해 증강 이미지 삭제 + 중복 이미지 삭제 
- Roboflow  
fresh, rotten으로 나누어진 이미지 중 모델 학습에 적합하다고 판단되는 데이터만 사용 + 중복 이미지 삭제

---

# Purpose
- ### 데이터의 다양성 확보
기존 데이터는 하나의 사과에 대해 여러 각도에서 찍은 여러 이미지 존재  
이 데이터를 증강시켜 사용  
⇒ 유사한 데이터가 많아 모델이 좀 더 일반적인 특성을 학습하기 어렵다고 판단  
- ### 각 클래스 별 데이터 불균형 해소
Apple_Fresh의 데이터가 Apple_Rotten 데이터에 비해 적음  
- ### 증강 누락 이미지 처리
1개의 이미지 당 2개의 증강 이미지 생성했으나 전체 데이터 수가 이전 데이터 수의 정확히 3배가 되지 않음  
증강 이미지의 파일명이 중복되어 일부 데이터 누락


# Difference
- Roboflow의 데이터 추가  
- Apple_Rotten 데이터 랜덤 언더샘플링
- 증강 이미지 파일명 : `f'aug_{filename.split(".")[0]}'` → `f'aug_{filename.split(".")[0]}_{str(uuid.uuid4())}'`

---

# Data Random Undersampling
## Data
|              |&nbsp;&nbsp;&nbsp;Original&nbsp;&nbsp;&nbsp;| Additional |&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Total&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|:------------:|:------------:|:------------:|:------------:|
| Apple_Fresh  | 368 | 92 | 460 |
| Apple_Rotten | 462 | 188 | 650 |  

Apple_Rotten : 총 650개의 이미지 중 190개의 이미지 랜덤 제거

In [1]:
import os
import random

In [2]:
directory = "/tf/Fixed_Data/Data_Final/Data/apple/stale"
delete_count = 190

In [3]:
image_files = [f for f in os.listdir(directory) if f.endswith((".jpg", ".png"))]

# 삭제할 파일 랜덤 선택
files_to_delete = random.sample(image_files, delete_count)

# 삭제된 파일 수
count = 0

# 파일 삭제
for file_name in files_to_delete:
    file_path = os.path.join(directory, file_name)
    os.remove(file_path)
    
    count += 1

In [4]:
print(f"deleted images: {count}")
print(f"current images: {len([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])}")

deleted images: 190
current images: 460


# Data Augmentation
`ImageDataGenerator` 사용

## Augmentation Ratio
1개의 이미지 당 2개의 증강 이미지 생성

## Data
|              |&nbsp;&nbsp;&nbsp;Original&nbsp;&nbsp;&nbsp;| Augmentation |&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Final&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|:------------:|:------------:|:------------:|:------------:|
| Apple_Fresh  | 460 | 920 | 1380 |
| Apple_Rotten | 460 | 920 | 1380 |



In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
import numpy as np
import uuid  # uuid 라이브러리 추가

2024-11-21 11:28:11.266124: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
datagen = ImageDataGenerator(
    rotation_range=90,            # 회전 각도 범위 [-90, +90]
    shear_range=0.3,              # 전단 변형(층밀림) 각도 [0, 0.3](radian):최대 약 17도
    zoom_range=[0.7, 1.3],        # 확대/축소 범위. 원본의 70% ~ 130%
    brightness_range=[0.2, 1.3],  # 밝기 조정 범위. 원본의 20% ~ 130%
    horizontal_flip=True,         # 좌우 반전
    vertical_flip=True,           # 상하 반전
)

### 증강 이미지 저장명 변경

In [7]:
def data_augmentation(input_dir, output_dir):
    
    for filename in os.listdir(input_dir):
        
        if filename.endswith(".jpg") or filename.endswith(".png"):
            
            img_path = os.path.join(input_dir, filename)
            img = load_img(img_path)  # 이미지 로드
            x = img_to_array(img)     # numpy 배열로 변환
            x = np.expand_dims(x, axis=0)  # 배치 차원 추가
            
            # 원본 이미지 저장
            original_output_path = os.path.join(output_dir, f'original_{filename}')
            img.save(original_output_path)  # 원본 이미지 저장
        
            # 증강 이미지 생성 및 저장
            i = 0
            # 증강 이미지 저장명 변경
#             for batch in datagen.flow(x, batch_size=1, save_to_dir=output_dir, save_prefix=f'aug_{filename.split(".")[0]}', save_format='jpg'):
            for batch in datagen.flow(x, batch_size=1, save_to_dir=output_dir, save_prefix=f'aug_{filename.split(".")[0]}_{str(uuid.uuid4())}', save_format='jpg'):
                i += 1
                if i >= 2:  # 하나의 이미지당 2개의 증강 이미지
                    break

In [8]:
base_dir = "/tf/Fixed_Data/Data_Final/Data"

In [9]:
# Apple_Fresh
input_dir = os.path.join(base_dir, 'apple/fresh')
output_dir = os.path.join(base_dir, 'Step1/Apple_Fresh')

data_augmentation(input_dir, output_dir)

In [10]:
# Apple_Rotten
input_dir = os.path.join(base_dir, 'apple/stale')
output_dir = os.path.join(base_dir, 'Step1/Apple_Rotten')

data_augmentation(input_dir, output_dir)

# Data Split
## Original & Augmented Data
/tf/Fixed_Data/Data_Final/Data/Step1  
│  
├── Apple_Fresh  
│  
└── Apple_Rotten  

## Splited Data
/tf/Fixed_Data/Data_Final/Step1  
│  
├── train  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Apple_Rotten  
│  
├── validation  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Apple_Rotten  
│  
└── test  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Apple_Rotten  

    
## Split Ratio
train : validation : test = 7 : 2 : 1

## Dataset
|              | &nbsp;&nbsp;&nbsp; train &nbsp;&nbsp;&nbsp; | validation | &nbsp;&nbsp;&nbsp;&nbsp; test &nbsp;&nbsp;&nbsp;&nbsp; |
|:------------:|:------------:|:------------:|:------------:|
| Apple_Fresh  | 965 | 276 | 139 |
| Apple_Rotten | 965 | 276 | 139 |
| Total | 1930 | 552 | 278 |

## Code
교재 Chapter 8-2 서브셋 저장 코드 참고


In [11]:
import shutil
import random

### Source : 교재 코드 8-6

In [12]:
# 파일 리스트를 서브셋으로 저장
def make_subset(subset_name, file_list):
    
    for file_path, category in file_list:
        
        dst_dir = os.path.join(new_base_dir, subset_name, category)
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy(file_path, os.path.join(dst_dir, os.path.basename(file_path)))

In [13]:
# 주어진 비율로 데이터 분할 후 서브셋 생성
def split_data(base_dir, train_ratio, val_ratio):
    
    # 클래스 디렉토리 수집
    class_dirs = [d for d in os.listdir(base_dir) 
                  if os.path.isdir(os.path.join(base_dir, d)) and not d.startswith('.')]
    
    train_data = 0
    val_data = 0
    test_data = 0
    
    # 데이터 분할
    for class_dir in class_dirs:
        
        class_path = os.path.join(base_dir, class_dir)
        
        # 파일 목록 수집
        files = [(os.path.join(class_path, f), class_dir) 
                 for f in os.listdir(class_path) if not f.startswith('.')]
        
        # 데이터 shuffle
        random.shuffle(files)

        # 데이터 분할
        total = len(files)
        train_index = int(total * train_ratio)
        val_index = train_index + int(total * val_ratio)

        train_files = files[:train_index]
        val_files = files[train_index:val_index]
        test_files = files[val_index:]

        # 서브셋 생성
        make_subset("train", train_files)
        make_subset("validation", val_files)
        make_subset("test", test_files)
        
        train_data += len(train_files)
        val_data += len(val_files)
        test_data += len(test_files)

    return train_data, val_data, test_data

In [14]:
original_dir = "/tf/Fixed_Data/Data_Final/Data/Step1"
new_base_dir = "/tf/Fixed_Data/Data_Final/Step1"

In [15]:
# train:validation:test = 7:2:1
train_ratio = 0.7
val_ratio = 0.2

In [16]:
# 데이터 분할 및 서브셋 생성
train_count, val_count, test_count = split_data(original_dir, train_ratio, val_ratio)

print(f"Train files: {train_count}, Validation files: {val_count}, Test files: {test_count}")

Train files: 1930, Validation files: 552, Test files: 278
